In [1]:
from dotenv import load_dotenv

from core.data_sources.clob import CLOBDataSource
import warnings

from core.notifiers import NotificationManager
from research_notebooks.notifiers.hummingbot_status_report import telegram_report

warnings.filterwarnings("ignore")


# Main class to access central limit order book connectors
clob = CLOBDataSource()

In [2]:
clob.load_candles_cache()
candles = []
for key, value in clob.candles_cache.items():
    if key[-1] == "1m" and key[0] == "binance_perpetual":
        candles.append(value)

In [37]:
import pandas as pd

def generate_tf_signal(dataframe: pd.DataFrame):
    df = dataframe.copy()
    df.ta.ema(length=20, append=True)
    df.ta.ema(length=200, append=True)
    df.ta.ema(length=500, append=True)
    # Generate EMA crossover signals
    df['signal'] = 0

    # Long signal: EMA_20 > EMA_200 and EMA_20 > EMA_500
    long_condition = (df['EMA_20'] > df['EMA_200']) & (df['EMA_20'] > df['EMA_500'])

    # Short signal: EMA_20 < EMA_200 and EMA_20 < EMA_500
    short_condition = (df['EMA_20'] < df['EMA_200']) & (df['EMA_20'] < df['EMA_500'])

    df.loc[long_condition, 'signal'] = 1
    df.loc[short_condition, 'signal'] = -1
    # Calculate signal intensity based on EMA divergence
    # Distance of EMA_20 from the average of EMA_200 and EMA_500
    ema_long_avg = (df['EMA_200'] + df['EMA_500']) / 2
    ema_divergence = df['EMA_20'] - ema_long_avg

    # Normalize intensity using rolling percentiles for adaptive scaling
    window = 3000  # Rolling window for percentile calculation
    df['ema_divergence_pct'] = ema_divergence.rolling(window).rank(pct=True)

    # Calculate intensity: 0 (neutral) to 1 (maximum strength)
    df['signal_intensity'] = 0.0

    # For long signals: intensity based on how much EMA_20 is above the long EMAs
    long_mask = df['signal'] == 1
    df.loc[long_mask, 'signal_intensity'] = df.loc[long_mask, 'ema_divergence_pct']

    # For short signals: intensity based on how much EMA_20 is below the long EMAs
    short_mask = df['signal'] == -1
    df.loc[short_mask, 'signal_intensity'] = 1 - df.loc[short_mask, 'ema_divergence_pct']

    # Calculate price range for potential grid levels
    df['price_range'] = df['high'].rolling(window).max() - df['low'].rolling(window).min()
    df['range_pct'] = df['price_range'] / df['close']
    return df, df['signal'].iloc[-1], df['signal_intensity'].iloc[-1], df['range_pct'].iloc[-1], df["close"].iloc[-1]


In [38]:
tf_signal_long = []
tf_signal_short = []
for candle in candles:
    try:
        df, signal, intensity, range_pct, last_price = generate_tf_signal(candle.data)
        if signal == 1 and intensity > 0.7 and range_pct > 0.01:
            tf_signal_long.append((candle.trading_pair, signal, intensity, range_pct, last_price))
        elif signal == -1 and intensity > 0.7 and range_pct > 0.01:
            tf_signal_short.append((candle.trading_pair, signal, intensity, range_pct, last_price))
    except Exception as e:
        print(f"Error processing {candle.trading_pair}: {e}")

Error processing TRADOOR-USDT: 'EMA_500'
Error processing ASTER-USDT: 'EMA_500'


In [40]:
long_df = pd.DataFrame(tf_signal_long, columns=['trading_pair', 'signal', 'intensity', 'range_pct', 'last_price']).sort_values(by='intensity', ascending=False)
short_df = pd.DataFrame(tf_signal_short, columns=['trading_pair', 'signal', 'intensity', 'range_pct', 'last_price']).sort_values(by='intensity', ascending=False)

In [42]:
top_3_long = long_df.head(3)
top_3_short = short_df.head(3)

In [46]:
top_3_long["start_price"] = top_3_long["last_price"] * (1 - 0.5 * top_3_long["range_pct"])
top_3_long["end_price"] = top_3_long["last_price"] * (1 + 1.5 * top_3_long["range_pct"])
top_3_long["limit_price"] = top_3_long["last_price"] * (1 - 0.7 * top_3_long["range_pct"])
top_3_long

,trading_pair,signal,intensity,range_pct,last_price,start_price,end_price,limit_price
5,TUT-USDT,1,1,0.14146278,0.07684,0.071405,0.093145,0.069231
10,CATI-USDT,1,1,0.10440252,0.0954,0.09042,0.11034,0.088428
32,BAS-USDT,1,0.99733333,0.26273911,0.018977,0.016484,0.026456,0.0154868


In [47]:
top_3_short["start_price"] = top_3_short["last_price"] * (1 + 0.5 * top_3_short["range_pct"])
top_3_short["end_price"] = top_3_short["last_price"] * (1 - 1.5 * top_3_short["range_pct"])
top_3_short["limit_price"] = top_3_short["last_price"] * (1 + 0.7 * top_3_short["range_pct"])
top_3_short

,trading_pair,signal,intensity,range_pct,last_price,start_price,end_price,limit_price
219,DRIFT-USDT,-1,0.97866667,0.36668588,0.8675,1.02655,0.39035,1.09017
178,YALA-USDT,-1,0.978,0.20628415,0.1464,0.1615,0.1011,0.16754
286,BTR-USDT,-1,0.977,0.23499382,0.12145,0.13572,0.07864,0.141428


In [59]:
from core.notifiers import NotificationManager, NotificationMessage
from dotenv import load_dotenv
import datetime
load_dotenv()

manager = NotificationManager()
notifier = manager.get_notifier("telegram")

# Format long signals
long_msg = ""
if not top_3_long.empty:
    for _, row in top_3_long.iterrows():
        long_msg += f"🟢 {row['trading_pair']}\n"
        long_msg += f"   💪 Intensity: {row['intensity']:.3f}\n"
        long_msg += f"   💰 Price: ${row['last_price']:.6f}\n"
        long_msg += f"   📊 Range: {row['range_pct']*100:.2f}%\n"
        long_msg += f"   🎯 Entry: ${row['limit_price']:.6f}\n"
        long_msg += f"   🚀 Target: ${row['end_price']:.6f}\n\n"
else:
    long_msg = "   No strong long signals found\n\n"

# Format short signals  
short_msg = ""
if not top_3_short.empty:
    for _, row in top_3_short.iterrows():
        short_msg += f"🔴 {row['trading_pair']}\n"
        short_msg += f"   💪 Intensity: {row['intensity']:.3f}\n"
        short_msg += f"   💰 Price: ${row['last_price']:.6f}\n"
        short_msg += f"   📊 Range: {row['range_pct']*100:.2f}%\n"
        short_msg += f"   🎯 Entry: ${row['limit_price']:.6f}\n"
        short_msg += f"   📉 Target: ${row['end_price']:.6f}\n\n"
else:
    short_msg = "   No strong short signals found\n\n"

current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M UTC")

telegram_report = f"""📈 TREND FOLLOWING SIGNALS
⏰ Time: {current_time}
📊 Timeframe: 1m
🎯 Min Intensity: 70%
📏 Min Range: 1%

🟢 LONG OPPORTUNITIES:
{long_msg}🔴 SHORT OPPORTUNITIES:
{short_msg}💡 Strategy: EMA(20) vs EMA(200/500) crossover
⚡ High intensity = stronger trend divergence
📊 Range % = recent volatility for grid sizing"""

notification = NotificationMessage(
    title="🤖 Trend Following Alert",
    message=telegram_report,
    level="info"
)

In [60]:
await notifier.send_notification(notification)

True

In [61]:
import yaml
from datetime import datetime

def generate_grid_config(trading_pair, side, start_price, limit_price, end_price):
    """
    Generate grid controller config
    side: 1 for long, 2 for short
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    direction = "long" if side == 1 else "short"
    config_id = f"tf_{trading_pair.lower().replace('-', '_')}_{direction}_{timestamp}"
    
    config = {
        'activation_bounds': '0.003',
        'connector_name': 'binance_perpetual',
        'controller_name': 'grid_strike',
        'controller_type': 'generic',
        'end_price': str(end_price),
        'id': config_id,
        'keep_position': True,
        'leverage': 20,
        'limit_price': str(limit_price),
        'max_open_orders': 10,
        'max_orders_per_batch': 2,
        'min_order_amount_quote': '10',
        'min_spread_between_orders': '0.002',
        'order_frequency': 5,
        'position_mode': 'HEDGE',
        'side': side,
        'start_price': str(start_price),
        'total_amount_quote': '400',
        'trading_pair': trading_pair,
        'triple_barrier_config': {
            'open_order_type': 3,
            'take_profit': '0.0035',
            'take_profit_order_type': 3
        }
    }
    
    return config

# Generate configs for top signals
all_configs = []

# Process long signals
for _, row in top_3_long.iterrows():
    config = generate_grid_config(
        trading_pair=row['trading_pair'],
        side=1,  # long
        start_price=row['start_price'],
        limit_price=row['limit_price'], 
        end_price=row['end_price']
    )
    all_configs.append(config)
    print(f"✅ Generated LONG config for {row['trading_pair']}")

# Process short signals  
for _, row in top_3_short.iterrows():
    config = generate_grid_config(
        trading_pair=row['trading_pair'],
        side=2,  # short
        start_price=row['start_price'],
        limit_price=row['limit_price'],
        end_price=row['end_price'] 
    )
    all_configs.append(config)
    print(f"✅ Generated SHORT config for {row['trading_pair']}")

print(f"\n📋 Total configs generated: {len(all_configs)}")

✅ Generated LONG config for TUT-USDT
✅ Generated LONG config for CATI-USDT
✅ Generated LONG config for BAS-USDT
✅ Generated SHORT config for DRIFT-USDT
✅ Generated SHORT config for YALA-USDT
✅ Generated SHORT config for BTR-USDT

📋 Total configs generated: 6


In [64]:
from hummingbot_api_client import HummingbotAPIClient

client = HummingbotAPIClient(base_url="http://localhost:8000")
await client.init()

In [70]:
for config in all_configs:
    await client.controllers.create_or_update_controller_config(config_name=config['id'], config=config)

In [72]:
await client.bot_orchestration.deploy_v2_controllers(
    instance_name="cohort-12-tf",
    credentials_profile="master_account",
    controllers_config=[config["id"] for config in all_configs],
)

{'success': True,
 'message': 'Instance cohort-12-tf created successfully.',
 'script_config_generated': 'cohort-12-tf-20250919-143723.yml',
 'controllers_deployed': ['tf_tut_usdt_long_20250919_1427',
  'tf_cati_usdt_long_20250919_1427',
  'tf_bas_usdt_long_20250919_1427',
  'tf_drift_usdt_short_20250919_1427',
  'tf_yala_usdt_short_20250919_1427',
  'tf_btr_usdt_short_20250919_1427']}

In [74]:
await client.bot_orchestration.get_active_bots_status()

{'status': 'success',
 'data': {'cohort-12-tf': {'status': 'running',
   'performance': {'tf_tut_usdt_long_20250919_1427': {'status': 'running',
     'performance': {'realized_pnl_quote': 0.0,
      'unrealized_pnl_quote': 4.81640947,
      'unrealized_pnl_pct': 0.44434712804699367,
      'realized_pnl_pct': 0.0,
      'global_pnl_quote': 4.81640947,
      'global_pnl_pct': 0.44434712804699367,
      'volume_traded': 1083.92947,
      'positions_summary': [],
      'close_type_counts': {}}},
    'tf_cati_usdt_long_20250919_1427': {'status': 'running',
     'performance': {'realized_pnl_quote': 0.0,
      'unrealized_pnl_quote': -11.27885728,
      'unrealized_pnl_pct': -1.4699635659726407,
      'realized_pnl_pct': 0.0,
      'global_pnl_quote': -11.27885728,
      'global_pnl_pct': -1.4699635659726407,
      'volume_traded': 767.28822,
      'positions_summary': [],
      'close_type_counts': {}}},
    'tf_bas_usdt_long_20250919_1427': {'status': 'running',
     'performance': {'reali

In [75]:
await client.bot_orchestration.stop_and_archive_bot(bot_name="cohort-12-tf")

{'status': 'success',
 'message': 'Stop and archive process started for bot cohort-12-tf',
 'details': {'input_name': 'cohort-12-tf',
  'actual_bot_name': 'cohort-12-tf',
  'container_name': 'cohort-12-tf',
  'process': 'The bot will be gracefully stopped, archived, and removed in the background. This process typically takes 20-30 seconds.'}}

In [77]:
await client.archived_bots.list_databases()

['bots/archived/cohort-12-tf/data/cohort-12-tf-20250919-143723.sqlite',
 'bots/archived/sapien_grid_bot/data/sapien_grid_bot-20250822-140441.sqlite',
 'bots/archived/myx_short_grid_bot/data/myx_short_grid_bot-20250804-183238.sqlite',
 'bots/archived/myx_short_grid_bot_v2/data/myx_short_grid_bot_v2-20250804-184503.sqlite']

In [78]:
db = "bots/archived/cohort-12-tf/data/cohort-12-tf-20250919-143723.sqlite"

await client.archived_bots.get_database_controllers(db)

{'db_path': 'bots/archived/cohort-12-tf/data/cohort-12-tf-20250919-143723.sqlite',
 'controllers': [{'id': 'tf_tut_usdt_long_20250919_1427',
   'controller_id': 1,
   'timestamp': 1758303447.511206,
   'type': 'generic',
   'config': '{"controller_name": "grid_strike", "controller_type": "generic", "total_amount_quote": "1000", "manual_kill_switch": false, "candles_config": [], "initial_positions": [], "leverage": 20, "position_mode": "HEDGE", "connector_name": "binance_perpetual", "trading_pair": "TUT-USDT", "side": 1, "start_price": "0.07140500000000001", "end_price": "0.09314500000000002", "limit_price": "0.069231", "min_spread_between_orders": "0.002", "min_order_amount_quote": "10", "max_open_orders": 10, "max_orders_per_batch": 2, "order_frequency": 5, "activation_bounds": "0.003", "keep_position": true, "triple_barrier_config": {"stop_loss": null, "take_profit": "0.0035", "time_limit": null, "trailing_stop": null, "open_order_type": 3, "take_profit_order_type": 3, "stop_loss_ord

In [79]:
await client.archived_bots.get_database_trades(db)

{'db_path': 'bots/archived/cohort-12-tf/data/cohort-12-tf-20250919-143723.sqlite',
 'trades': [{'config_file_path': 'cohort-12-tf-20250919-143723.yml',
   'strategy': 'v2_with_controllers',
   'connector_name': 'binance_perpetual',
   'trading_pair': 'CATI-USDT',
   'base_asset': 'CATI',
   'quote_asset': 'USDT',
   'timestamp': 1758303459000,
   'order_id': 'x-nbQe1H39BCIUT63f2aecfeaa8f9293',
   'trade_type': 'BUY',
   'order_type': 'LIMIT_MAKER',
   'price': 0.09865,
   'amount': 107.0,
   'leverage': 20,
   'trade_fee': 0.0,
   'trade_fee_in_quote': 0.002111,
   'exchange_trade_id': '93398168',
   'position': 'OPEN',
   'cum_fees_in_quote': 0.002111},
  {'config_file_path': 'cohort-12-tf-20250919-143723.yml',
   'strategy': 'v2_with_controllers',
   'connector_name': 'binance_perpetual',
   'trading_pair': 'CATI-USDT',
   'base_asset': 'CATI',
   'quote_asset': 'USDT',
   'timestamp': 1758303459000,
   'order_id': 'x-nbQe1H39BCIUT63f2aecfea9b79293',
   'trade_type': 'BUY',
   'order